In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from utils import *
import logging
import json

In [3]:
from gensim.corpora.dictionary import Dictionary
import gensim
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.linear_model import *
import scipy.sparse as sps
from sklearn.model_selection import KFold
from gensim.models.tfidfmodel import TfidfModel
import numpy as np

In [4]:
from itertools import chain
from sklearn import linear_model
from sklearn import model_selection

In [4]:
logging.basicConfig(level=logging.INFO)

In [5]:

fname = "data/train.csv"
data = pd.read_csv(fname, header=0)
head = data.head().copy()


In [8]:
work_data = data.copy()

In [9]:
work_data.question1 = data.question1.apply(normalize)
work_data.question2 = data.question2.apply(normalize)

INFO:root:600000
INFO:root:700000
INFO:root:800000
INFO:root:900000
INFO:root:1000000
INFO:root:1100000
INFO:root:1200000
INFO:root:1300000


In [10]:
pom = pd.concat([work_data.question1, work_data.question2])
dic = Dictionary(pom)
dic.filter_extremes(no_below=3, no_above=0.1)

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:adding document #10000 to Dictionary(11705 unique tokens: ['installing', 'which', '5ft', 'mla', 'combustible']...)
INFO:gensim.corpora.dictionary:adding document #20000 to Dictionary(16732 unique tokens: ['suicides', 'nightclubs', 'installing', 'euntrepreneur', 'which']...)
INFO:gensim.corpora.dictionary:adding document #30000 to Dictionary(20448 unique tokens: ['suicides', 'nightclubs', 'installing', 'euntrepreneur', 'resetting']...)
INFO:gensim.corpora.dictionary:adding document #40000 to Dictionary(23451 unique tokens: ['racing', 'library', 'bombay', 'realistic', 'mla']...)
INFO:gensim.corpora.dictionary:adding document #50000 to Dictionary(26283 unique tokens: ['racing', 'library', 'bombay', 'ziffi', 'icmr']...)
INFO:gensim.corpora.dictionary:adding document #60000 to Dictionary(28645 unique tokens: ['racing', 'library', 'bombay', 'ziffi', 'icmr']...)
INFO:gensim.corp

In [11]:
work_data.question1 = work_data.question1.apply(lambda x: dic.doc2bow(x))
work_data.question2 = work_data.question2.apply(lambda x: dic.doc2bow(x))

In [5]:
work_data.to_pickle("data/train_vectorized.pickle")
dic.save("data/dictionary.dic")

NameError: name 'work_data' is not defined

In [ ]:
sklearn.model_selection.GridSearchCV()

In [6]:
work_data = pd.read_pickle("data/train_vectorized.pickle")
dic = gensim.corpora.Dictionary.load("data/dictionary.dic")

INFO:gensim.utils:loading Dictionary object from data/dictionary.dic
INFO:gensim.utils:loaded data/dictionary.dic


In [7]:
tfidf = TfidfModel(pd.concat([work_data.question1, work_data.question2]))

INFO:gensim.models.tfidfmodel:collecting document frequencies
INFO:gensim.models.tfidfmodel:PROGRESS: processing document #0
INFO:gensim.models.tfidfmodel:PROGRESS: processing document #10000
INFO:gensim.models.tfidfmodel:PROGRESS: processing document #20000
INFO:gensim.models.tfidfmodel:PROGRESS: processing document #30000
INFO:gensim.models.tfidfmodel:PROGRESS: processing document #40000
INFO:gensim.models.tfidfmodel:PROGRESS: processing document #50000
INFO:gensim.models.tfidfmodel:PROGRESS: processing document #60000
INFO:gensim.models.tfidfmodel:PROGRESS: processing document #70000
INFO:gensim.models.tfidfmodel:PROGRESS: processing document #80000
INFO:gensim.models.tfidfmodel:PROGRESS: processing document #90000
INFO:gensim.models.tfidfmodel:PROGRESS: processing document #100000
INFO:gensim.models.tfidfmodel:PROGRESS: processing document #110000
INFO:gensim.models.tfidfmodel:PROGRESS: processing document #120000
INFO:gensim.models.tfidfmodel:PROGRESS: processing document #130000


In [8]:
work_data.question1 = work_data.question1.apply(lambda x: tfidf[x])
work_data.question2 = work_data.question2.apply(lambda x: tfidf[x])

In [9]:
work_data.question2.head()[0]

[(4307, 0.3104637458592939),
 (31103, 0.3249384776224683),
 (36323, 0.3953716277550753),
 (36609, 0.19369471786822115),
 (36728, 0.29112713390459066),
 (37302, 0.7207189968414592)]

In [10]:
q1 = gensim.matutils.corpus2csc(work_data.question1).transpose()
q2 = gensim.matutils.corpus2csc(work_data.question2).transpose()
q1 = sps.csr_matrix((q1.data, q1.indices, q1.indptr), shape=(work_data.shape[0], max(dic.dfs.keys())+1))
q2 = sps.csr_matrix((q2.data, q2.indices, q2.indptr), shape=(work_data.shape[0], max(dic.dfs.keys())+1))

In [11]:
q1.shape

(404290, 39649)

In [15]:
X = q1.multiply(q2)
X = X.sqrt()
normalizer = (((q1+q2)>0).sum(axis=1)+1)*0 + 1
row_indices, col_indices = X.nonzero()
X.data /= normalizer[row_indices].A1

In [16]:
Y = work_data.is_duplicate

In [17]:
from sklearn.neural_network import MLPClassifier

In [22]:
clsClass = MLPClassifier

In [25]:
kf = KFold(n_splits=2)
#clf = LogisticRegression(class_weight=None, max_iter=100, verbose=100, warm_start=False, n_jobs=4)
clf = clsClass(verbose=True, max_iter=10)
for train_index, test_index in kf.split(X):
    clf.fit(X[train_index],Y[train_index])
    probs = clf.predict_proba(X[test_index])[:,1]
    print(loss(Y[test_index], probs))

Iteration 1, loss = 0.52417773
Iteration 2, loss = 0.44734636
Iteration 3, loss = 0.41202244
Iteration 4, loss = 0.38647826
Iteration 5, loss = 0.36548581
Iteration 6, loss = 0.34630563
Iteration 7, loss = 0.32869272
Iteration 8, loss = 0.31193208
Iteration 9, loss = 0.29580378
Iteration 10, loss = 0.28021747


/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


0.526889424452
Iteration 1, loss = 0.52301382
Iteration 2, loss = 0.44414232
Iteration 3, loss = 0.40734260
Iteration 4, loss = 0.37996068
Iteration 5, loss = 0.35684377
Iteration 6, loss = 0.33539139
Iteration 7, loss = 0.31566567
Iteration 8, loss = 0.29699220
Iteration 9, loss = 0.27882285
Iteration 10, loss = 0.26269357
0.532677161973


In [23]:
fname = "data/test.csv"
test = pd.read_csv(fname, header=0)

KeyboardInterrupt: 

In [14]:
work_test = test.copy()
work_test.question1 = test.question1.apply(normalize)
work_test.question2 = test.question2.apply(normalize)

INFO:root:900000
INFO:root:1000000
INFO:root:1100000
INFO:root:1200000
INFO:root:1300000
INFO:root:1400000
INFO:root:1500000
INFO:root:1600000
INFO:root:1700000
INFO:root:1800000
INFO:root:1900000
INFO:root:2000000
INFO:root:2100000
INFO:root:2200000
INFO:root:2300000
INFO:root:2400000
INFO:root:2500000
INFO:root:2600000
INFO:root:2700000
INFO:root:2800000
INFO:root:2900000
INFO:root:3000000
INFO:root:3100000
INFO:root:3200000
INFO:root:3300000
INFO:root:3400000
INFO:root:3500000
INFO:root:3600000
INFO:root:3700000
INFO:root:3800000
INFO:root:3900000
INFO:root:4000000
INFO:root:4100000
INFO:root:4200000
INFO:root:4300000
INFO:root:4400000
INFO:root:4500000
INFO:root:4600000
INFO:root:4700000
INFO:root:4800000
INFO:root:4900000
INFO:root:5000000
INFO:root:5100000
INFO:root:5200000
INFO:root:5300000
INFO:root:5400000
INFO:root:5500000


In [25]:
work_test.to_pickle("data/cleaned.pickle")

In [24]:
work_test = pd.read_pickle("data/cleaned.pickle")

KeyboardInterrupt: 

In [29]:
X.shape

(404290, 39649)

In [20]:
test = []

In [21]:
vectorized_test = work_test
vectorized_test.question1 = work_test.question1.apply(lambda x: dic.doc2bow(x))
vectorized_test.question2 = work_test.question2.apply(lambda x: dic.doc2bow(x))

In [22]:
vectorized_test.to_pickle("data/vectorized_test.pickle")

In [25]:
vectorized_test = pd.read_pickle("data/vectorized_test.pickle")

In [26]:
vectorized_test.question1 = vectorized_test.question1.apply(lambda x: tfidf[x])
vectorized_test.question2 = vectorized_test.question2.apply(lambda x: tfidf[x])


In [30]:
vectorized_test.to_pickle("data/vectorized_tfidf_test.pickle")

In [19]:
vectorized_test = pd.read_pickle("data/vectorized_tfidf_test.pickle")

In [20]:
Q1 = gensim.matutils.corpus2csc(vectorized_test.question1).transpose()
Q2 = gensim.matutils.corpus2csc(vectorized_test.question2).transpose()
Q1 = sps.csr_matrix((Q1.data, Q1.indices, Q1.indptr), shape=(vectorized_test.shape[0], max(dic.dfs.keys())+1))
Q2 = sps.csr_matrix((Q2.data, Q2.indices, Q2.indptr), shape=(vectorized_test.shape[0], max(dic.dfs.keys())+1))

In [21]:
test_X = Q1.multiply(Q2)
test_X[test_X>1] = 1

test_X = Q1.multiply(Q2)
test_X[test_X>0] = 1
test_X = test_X.sqrt()
normalizer = (((Q1+Q2)>0).sum(axis=1)+1)*0+1
row_indices, col_indices = test_X.nonzero()
test_X.data /= normalizer[row_indices].A1

In [23]:
cls = clsClass(verbose=True, max_iter=30)

In [24]:
clf.fit(X, Y)

/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:565: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=True, warm_start=False)

In [34]:
test_probs = clf.predict_proba(test_X)

In [35]:
sub = pd.DataFrame({'test_id': vectorized_test['test_id'], 'is_duplicate': test_probs[:,1]})
sub.to_csv('multilayer_submission.csv', index=False)
sub.head()

,is_duplicate,test_id
0,0.026108,0
1,0.285429,1
2,0.408960,2
3,0.007848,3
4,0.325358,4


In [36]:
sub.head()

,is_duplicate,test_id
0,0.026108,0
1,0.285429,1
2,0.408960,2
3,0.007848,3
4,0.325358,4


In [1]:
data = pd
data.wob = data.question1
same = [[w for w in x[3] if w in x[4]] for x in data.values]
diff = [[w for w in x[3] if w not in x[4]] + [w for w in x[4] if w not in x[3]] for x in data.values]

NameError: name 'data' is not defined